In [15]:
import json

import agama
import h5py
import halo_analysis as halo
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.io import ascii
from gc_utils import iteration_name, main_prog_halt, snapshot_name  # type: ignore

In [16]:
simulation = "m12i"

sim_dir = "/Users/z5114326/Documents/simulations/"
data_dir = "/Users/z5114326/Documents/GitHub/gc_kinematics/data/"

fire_dir = sim_dir + simulation + "/" + simulation + "_res7100/"

sim = simulation

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "a")  # open processed data file

In [291]:
def get_birth_details(sim: str, it: int, sim_dir: str, data_dir: str, birth_dict={}):
    proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
    proc_data = h5py.File(proc_file, "r")  # open processed data file

    it_id = iteration_name(it)
    source_dat = proc_data[it_id]["source"]

    mask = (np.array(source_dat["group_id"]) == 0) & (np.array(source_dat["analyse_flag"]) == 1)
    gc_id = np.array(source_dat["gc_id"][mask])
    t_form = np.array(source_dat["form_time"][mask])
    t_disr = np.array(proc_data["it000"]["source"]["t_dis"][mask])

    all_snapshot_fil = sim_dir + sim + "/" + sim + "_res7100/snapshot_times.txt"
    with open(all_snapshot_fil) as f:
        content = f.readlines()
        content = content[5:]
    snap_all = ascii.read(content)["i"]
    tim_all = ascii.read(content)["time[Gyr]"]

    potential_snaps = data_dir + "external/potentials.json"
    with open(potential_snaps) as json_file:
        pot_data = json.load(json_file)

    pot_snap_lst = pot_data[sim]
    pot_time_lst = []
    for pot_snap in pot_snap_lst:
        idx = np.where(snap_all == pot_snap)[0][0]
        pot_time_lst.append(tim_all[idx])

    t_form_ref_a = []
    t_form_ref_b = []

    pot_ref_a = []
    pot_ref_b = []

    t_dif_prop_a = []
    t_dif_prop_b = []

    for t_f in t_form:
        filtered_a = [val for val in pot_time_lst if val >= t_f]
        time_ref_a = min(filtered_a, default=None)

        filtered_b = [val for val in pot_time_lst if val < t_f]
        if len(filtered_b) == 0:
            time_ref_b = 46
        else:
            time_ref_b = max(filtered_b, default=None)

        # filtered = [val for val in pot_time_lst if val >= t_f]
        # time_ref_a = min(filtered, default=None)
        # t_form_ref_a.append(time_ref_a)

        # time_ref_b = min(pot_time_lst, key=lambda x: abs(x - t_f))
        # t_form_ref_b.append(time_ref_b)

        pot_idx_a = np.where(pot_time_lst == time_ref_a)[0][0]
        pot_ref_a.append(pot_snap_lst[pot_idx_a])

        pot_idx_b = np.where(pot_time_lst == time_ref_b)[0][0]
        pot_ref_b.append(pot_snap_lst[pot_idx_b])

        t_dif_prop_a.append(t_f - time_ref_a)
        t_dif_prop_b.append(t_f - time_ref_b)

    birth_dict["gc_id"] = []
    birth_dict["radius"] = []
    birth_dict["log_mass"] = []
    birth_dict["r_apo"] = []
    birth_dict["r_per"] = []
    birth_dict["ecc"] = []
    birth_dict["t_disrupt"] = []
    birth_dict["t_birth"] = []

    failed_count = 0
    total_count = len(gc_id)
    for gc, snap_a, snap_b, t_dif_a, t_dif_b, t_d in zip(
        gc_id, pot_ref_a, pot_ref_b, t_dif_prop_a, t_dif_prop_b, t_disr
    ):
        # for gc, snap_b, t_dif_a in zip(gc_id, pot_ref_b, t_dif_prop_b)

        t_dif_lst = [t_dif_a, t_dif_b]
        snap_lst = [snap_a, snap_b]

        min_dif_idx = np.where(np.abs(t_dif_lst) == np.min(np.abs(t_dif_lst)))[0][0]
        max_dif_idx = 1 - min_dif_idx

        try:
            snap = snap_lst[min_dif_idx]
            t_dif = t_dif_lst[min_dif_idx]
            snap_id = snapshot_name(snap)
            snap_dict = proc_data[it_id]["snapshots"][snap_id]
            gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]
            pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)

        except:  # noqa: E722
            try:
                snap = snap_lst[max_dif_idx]
                t_dif = t_dif_lst[max_dif_idx]
                snap_id = snapshot_name(snap)
                snap_dict = proc_data[it_id]["snapshots"][snap_id]
                gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]
                pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
            except:  # noqa: E722
                try:
                    snap = snap_lst[max_dif_idx]
                    snap_idx = np.where(np.array(pot_snap_lst) == snap)[0][0] - 1
                    snap = pot_snap_lst[snap_idx]
                    snap_id = snapshot_name(snap)
                    snap_dict = proc_data[it_id]["snapshots"][snap_id]
                    gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]

                    t_dif = t_d - pot_time_lst[snap_idx]
                    pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
                except:  # noqa: E722
                    failed_count += 1
                    continue

        pos = [snap_dict["x"][gc_snap_idx], snap_dict["y"][gc_snap_idx], snap_dict["z"][gc_snap_idx]]
        vel = [snap_dict["vx"][gc_snap_idx], snap_dict["vy"][gc_snap_idx], snap_dict["vz"][gc_snap_idx]]
        ic = pos + vel

        agama.setUnits(mass=1, length=1, velocity=1)
        pot_nbody = agama.Potential(pot_file)

        # time is in Gyr, need to convert to time units of AGAMA: 1 time unit = 977.79222Myr
        t_prop_gyr = t_dif * u.Gyr
        agama_t_unit = agama.getUnits()["time"]
        t_prop_agama = t_prop_gyr.to(agama_t_unit)
        t_prop_agama = t_prop_agama.value

        if t_prop_agama == 0:
            pc = ic
            radii_pos = pot_nbody.Rperiapo(pc)
            radius = np.linalg.norm(pc[:3])
        else:
            _, pc = agama.orbit(ic=ic, potential=pot_nbody, time=t_prop_agama, trajsize=1)
            radii_pos = pot_nbody.Rperiapo(pc)[0]
            radius = np.linalg.norm(pc[0][:3])

        # print(radii_pos)
        r_per = radii_pos[0]
        r_apo = radii_pos[1]

        eccentricity = (r_apo - r_per) / (r_apo + r_per)

        gc_source_idx = np.where(gc_id == gc)[0][0]
        log_mass = source_dat["logm_tform"][gc_source_idx]

        birth_dict["gc_id"].append(gc)
        birth_dict["radius"].append(radius)
        birth_dict["log_mass"].append(log_mass)
        birth_dict["r_apo"].append(r_apo)
        birth_dict["r_per"].append(r_per)
        birth_dict["ecc"].append(eccentricity)
        birth_dict["t_disrupt"].append(t_d)
        birth_dict["t_birth"].append(source_dat["form_time"][gc_source_idx])

    print("success:", total_count - failed_count, "/", total_count)
    return birth_dict

In [292]:
def get_death_details(sim: str, it: int, sim_dir: str, data_dir: str, death_dict={}):
    proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
    proc_data = h5py.File(proc_file, "r")  # open processed data file

    it_id = iteration_name(it)
    source_dat = proc_data[it_id]["source"]

    mask = (np.array(source_dat["group_id"]) == 0) & (np.array(source_dat["analyse_flag"]) == 1)
    gc_id = np.array(source_dat["gc_id"][mask])
    t_disr = np.array(source_dat["t_dis"][mask])

    all_snapshot_fil = sim_dir + sim + "/" + sim + "_res7100/snapshot_times.txt"
    with open(all_snapshot_fil) as f:
        content = f.readlines()
        content = content[5:]
    snap_all = ascii.read(content)["i"]
    tim_all = ascii.read(content)["time[Gyr]"]

    potential_snaps = data_dir + "external/potentials.json"
    with open(potential_snaps) as json_file:
        pot_data = json.load(json_file)

    pot_snap_lst = pot_data[sim]
    pot_time_lst = []
    for pot_snap in pot_snap_lst:
        idx = np.where(snap_all == pot_snap)[0][0]
        pot_time_lst.append(tim_all[idx])

    t_disr_ref_a = []
    t_disr_ref_b = []

    pot_ref_a = []
    pot_ref_b = []

    t_dif_prop_a = []
    t_dif_prop_b = []

    gc_corr_lst = []
    for t_d, gc in zip(t_disr, gc_id):
        if t_d == -1:
            # cluster survives
            continue
        gc_corr_lst.append(gc)

        filtered_a = [val for val in pot_time_lst if val >= t_d]
        time_ref_a = min(filtered_a, default=None)
        # t_disr_ref_a.append(time_ref_a)

        filtered_b = [val for val in pot_time_lst if val < t_d]
        time_ref_b = max(filtered_b, default=None)
        # t_disr_ref_b.append(time_ref_b)

        # time_ref_b = min(pot_time_lst, key=lambda x: abs(x - t_d))
        # t_disr_ref_b.append(time_ref_b)

        pot_idx_a = np.where(pot_time_lst == time_ref_a)[0][0]
        pot_ref_a.append(pot_snap_lst[pot_idx_a])

        pot_idx_b = np.where(pot_time_lst == time_ref_b)[0][0]
        pot_ref_b.append(pot_snap_lst[pot_idx_b])

        t_dif_prop_a.append(t_d - time_ref_a)
        t_dif_prop_b.append(t_d - time_ref_b)

    death_dict["gc_id"] = []
    death_dict["radius"] = []
    death_dict["r_apo"] = []
    death_dict["r_per"] = []
    death_dict["ecc"] = []
    death_dict["t_disrupt"] = []

    failed_count = 0
    total_count = len([t_d for t_d in t_disr if t_d != -1])
    for gc, snap_a, snap_b, t_dif_a, t_dif_b, t_d in zip(
        gc_corr_lst, pot_ref_a, pot_ref_b, t_dif_prop_a, t_dif_prop_b, t_disr
    ):
        # for gc, snap_b, t_dif_a in zip(gc_id, pot_ref_b, t_dif_prop_b)

        t_dif_lst = [t_dif_a, t_dif_b]
        snap_lst = [snap_a, snap_b]

        min_dif_idx = np.where(np.abs(t_dif_lst) == np.min(np.abs(t_dif_lst)))[0][0]
        max_dif_idx = 1 - min_dif_idx

        try:
            snap = snap_lst[min_dif_idx]
            t_dif = t_dif_lst[min_dif_idx]
            snap_id = snapshot_name(snap)
            snap_dict = proc_data[it_id]["snapshots"][snap_id]
            gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]
            pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)

        except:  # noqa: E722
            try:
                snap = snap_lst[max_dif_idx]
                t_dif = t_dif_lst[max_dif_idx]
                snap_id = snapshot_name(snap)
                snap_dict = proc_data[it_id]["snapshots"][snap_id]
                gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]
                pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
            except:  # noqa: E722
                try:
                    # I have to include this stupid second esception because the last snap is often not aligning with
                    # the time of disruption, because of the way Chen has constructed his system. For example, it may
                    # say that it was disrupted after snap 102 but it has no mass at snap 102.
                    snap = snap_lst[min_dif_idx]
                    snap_idx = np.where(np.array(pot_snap_lst) == snap)[0][0] - 1
                    snap = pot_snap_lst[snap_idx]
                    snap_id = snapshot_name(snap)
                    snap_dict = proc_data[it_id]["snapshots"][snap_id]
                    gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]

                    t_dif = t_d - pot_time_lst[snap_idx]
                    pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
                except:  # noqa: E722
                    failed_count += 1
                    continue

        pos = [snap_dict["x"][gc_snap_idx], snap_dict["y"][gc_snap_idx], snap_dict["z"][gc_snap_idx]]
        vel = [snap_dict["vx"][gc_snap_idx], snap_dict["vy"][gc_snap_idx], snap_dict["vz"][gc_snap_idx]]
        ic = pos + vel

        agama.setUnits(mass=1, length=1, velocity=1)
        pot_nbody = agama.Potential(pot_file)

        # time is in Gyr, need to convert to time units of AGAMA: 1 time unit = 977.79222Myr
        t_prop_gyr = t_dif * u.Gyr
        agama_t_unit = agama.getUnits()["time"]
        t_prop_agama = t_prop_gyr.to(agama_t_unit)
        t_prop_agama = t_prop_agama.value

        if t_prop_agama == 0:
            pc = ic
            radii_pos = pot_nbody.Rperiapo(pc)
            radius = np.linalg.norm(pc[:3])
        else:
            _, pc = agama.orbit(ic=ic, potential=pot_nbody, time=t_prop_agama, trajsize=1)
            radii_pos = pot_nbody.Rperiapo(pc)[0]
            radius = np.linalg.norm(pc[0][:3])

        # print(radii_pos)
        r_per = radii_pos[0]
        r_apo = radii_pos[1]

        eccentricity = (r_apo - r_per) / (r_apo + r_per)

        death_dict["gc_id"].append(gc)
        death_dict["radius"].append(radius)
        death_dict["r_apo"].append(r_apo)
        death_dict["r_per"].append(r_per)
        death_dict["ecc"].append(eccentricity)
        death_dict["t_disrupt"].append(t_d)

    print("success:", total_count - failed_count, "/", total_count)
    return death_dict

In [293]:
data_dict = {}

# There are some failures as they are born and die between public snapshots so can't get any information.
# might need to get gizmo parts to do this (sign) if can be arsed.

birth_dict = get_birth_details(sim, 0, sim_dir, data_dir, birth_dict={})
death_dict = get_death_details(sim, 0, sim_dir, data_dir, death_dict={})

data_dict["birth"] = birth_dict
data_dict["death"] = death_dict

# success: 8370 / 8764
# success: 8222 / 8616

/var/folders/nb/byfjq48d6jz8b3g74s3zr7rh0000gp/T/ipykernel_9845/3299903394.py:59: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  pot_idx_b = np.where(pot_time_lst == time_ref_b)[0][0]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [267]:
proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

it = 0
it_id = iteration_name(it)
source_dat = proc_data[it_id]["source"]

mask = (np.array(source_dat["group_id"]) == 0) & (np.array(source_dat["analyse_flag"]) == 1)
gc_id = np.array(source_dat["gc_id"][mask])
t_form = np.array(source_dat["form_time"][mask])
t_disr = np.array(source_dat["t_dis"][mask])

all_snapshot_fil = sim_dir + sim + "/" + sim + "_res7100/snapshot_times.txt"
with open(all_snapshot_fil) as f:
    content = f.readlines()
    content = content[5:]
snap_all = ascii.read(content)["i"]
tim_all = ascii.read(content)["time[Gyr]"]

potential_snaps = data_dir + "external/potentials.json"
with open(potential_snaps) as json_file:
    pot_data = json.load(json_file)

pot_snap_lst = pot_data[sim]
pot_time_lst = []
for pot_snap in pot_snap_lst:
    idx = np.where(snap_all == pot_snap)[0][0]
    pot_time_lst.append(tim_all[idx])

t_disr_ref_a = []
t_disr_ref_b = []
pot_ref_a = []
pot_ref_b = []
t_dif_prop_a = []
t_dif_prop_b = []

gc_corr_lst = []

for t_d, gc in zip(t_disr, gc_id):
    if t_d == -1:
        # cluster survives
        continue
    gc_corr_lst.append(gc)

    filtered_a = [val for val in pot_time_lst if val >= t_d]
    time_ref_a = min(filtered_a, default=None)
    t_disr_ref_a.append(time_ref_a)

    filtered_b = [val for val in pot_time_lst if val < t_d]
    time_ref_b = max(filtered_b, default=None)
    t_disr_ref_b.append(time_ref_b)

    # time_ref_b = min(pot_time_lst, key=lambda x: abs(x - t_d))
    # t_disr_ref_b.append(time_ref_b)

    pot_idx_a = np.where(pot_time_lst == time_ref_a)[0][0]
    pot_ref_a.append(pot_snap_lst[pot_idx_a])

    pot_idx_b = np.where(pot_time_lst == time_ref_b)[0][0]
    pot_ref_b.append(pot_snap_lst[pot_idx_b])

    t_dif_prop_a.append(t_d - time_ref_a)
    t_dif_prop_b.append(t_d - time_ref_b)

death_dict["gc_id"] = []
death_dict["radius"] = []
death_dict["r_apo"] = []
death_dict["r_per"] = []
death_dict["ecc"] = []
death_dict["t_disrupt"] = []

In [275]:
# for gc, snap_a, snap_b, t_dif_a, t_dif_b, t_d in zip(
#         gc_id, pot_ref_a, pot_ref_b, t_dif_prop_a, t_dif_prop_b, t_disr
#     ):

gc = 49473733


test_idx = np.where(np.array(gc_corr_lst) == gc)[0][0]
snap_a = pot_ref_a[test_idx]
snap_b = pot_ref_b[test_idx]

t_dif_a = t_dif_prop_a[test_idx]
t_dif_b = t_dif_prop_b[test_idx]

t_dif_lst = [t_dif_a, t_dif_b]
snap_lst = [snap_a, snap_b]

min_dif_idx = np.where(t_dif_lst == np.min(t_dif_lst))[0][0]
max_dif_idx = 1 - min_dif_idx

# snap = snap_lst[min_dif_idx]
# t_dif = t_dif_lst[min_dif_idx]

# snap = snap_lst[max_dif_idx]
# t_dif = t_dif_lst[max_dif_idx]

t_dif_lst = [t_dif_a, t_dif_b]
snap_lst = [snap_a, snap_b]

min_dif_idx = np.where(np.abs(t_dif_lst) == np.min(np.abs(t_dif_lst)))[0][0]
max_dif_idx = 1 - min_dif_idx

try:
    snap = snap_lst[min_dif_idx]
    t_dif = t_dif_lst[min_dif_idx]
    snap_id = snapshot_name(snap)
    snap_dict = proc_data[it_id]["snapshots"][snap_id]
    gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]
    pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)

except:
    try:
        snap = snap_lst[max_dif_idx]
        t_dif = t_dif_lst[max_dif_idx]
        snap_id = snapshot_name(snap)
        snap_dict = proc_data[it_id]["snapshots"][snap_id]
        gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]
        pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
    except:
        try:
            # I have to include this stupid second esception because the last snap is often not aligning with
            # the time of disruption, because of the way Chen has constructed his system. For example, it may
            # say that it was disrupted after snap 102 but it has no mass at snap 102.

            # we choose max here as this is the time step before t_disrupt and we want to goi one step earlier
            snap = snap_lst[max_dif_idx]
            snap_idx = np.where(np.array(pot_snap_lst) == snap)[0][0] - 1
            snap = pot_snap_lst[snap_idx]
            snap_id = snapshot_name(snap)
            snap_dict = proc_data[it_id]["snapshots"][snap_id]

            gc_idx = np.where(np.array(gc_id) == gc)[0][0]
            t_d = t_disr[gc_idx]
            t_dif = t_d - pot_time_lst[snap_idx]
            pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)

            gc_snap_idx = np.where(np.array(snap_dict["gc_id"]) == gc)[0][0]

        except:
            print("eat shit finn")

eat shit finn


In [276]:
snap_lst, snap, t_form[gc_idx], t_d

([277, 214], 214, 3.465985274, 4.392)

In [277]:
idx_gc = np.where(
    (np.array(proc_data[it_id]["source"]["gc_id"]) == gc)
    & (np.array(proc_data[it_id]["source"]["analyse_flag"]) == 1)
)[0][0]
idx_gc

6084

In [278]:
(
    proc_data[it_id]["source"]["form_time"][idx_gc],
    proc_data[it_id]["source"]["pubsnap_zform"][idx_gc],
    proc_data[it_id]["source"]["last_snap"][idx_gc],
    proc_data[it_id]["source"]["t_dis"][idx_gc],
)

(3.465985274, 214, 214, 4.392)

In [279]:
np.where(np.array(proc_data[it_id]["snapshots"]["snap088"]["gc_id"]) == gc)

(array([], dtype=int64),)